In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [4]:
required_env_vars = [
    "NEO4J_URI",
    "NEO4J_USERNAME",
    "NEO4J_PASSWORD",
    "OPENAI_API_KEY",
    "ANTHROPIC_API_KEY"
]

for var in required_env_vars:
    assert os.getenv(var) , f"Environment variable {var} is not set."

#### We will be using anthropic API for graph creation since it gives the best results, and OpenAI for inference since it is faster and cheaper.

In [5]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

In [6]:
llm_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0,
    max_tokens=4096,
    max_retries=2
)

In [7]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [8]:
system_prompt = (
    "# Knowledge Graph Instructions\n"
    "## 1. Overview\n"
    "You are a top-tier algorithm designed for extracting information in structured "
    "formats to build a knowledge graph.\n"
    "Try to capture as much information from the text as possible without "
    "sacrificing accuracy. Do not add any information that is not explicitly "
    "mentioned in the text.\n"
    "- **Nodes** represent entities and concepts.\n"
    "- The aim is to achieve simplicity and clarity in the knowledge graph, making it\n"
    "accessible for a vast audience.\n"
    "## 2. Labeling Nodes\n"
    "- **Consistency**: Ensure you use available types for node labels.\n"
    "Ensure you use basic or elementary types for node labels.\n"
    "- For example, when you identify an entity representing a person, "
    "always label it as **'person'**. Avoid using more specific terms "
    "like 'mathematician' or 'scientist'."
    "- **Node IDs**: Never utilize integers as node IDs. Node IDs should be "
    "names or human-readable identifiers found in the text.\n"
    "- **Node Names**: Create a **name** property for each node it should be names, or human-readable identifiers found in the text.\n"
    "- **Relationships** represent connections between entities or concepts.\n"
    "Ensure consistency and generality in relationship types when constructing "
    "knowledge graphs. Instead of using specific and momentary types "
    "such as 'BECAME_PROFESSOR', use more general and timeless relationship types "
    "like 'PROFESSOR'. Make sure to use general and timeless relationship types!\n"
    "## 3. Coreference Resolution\n"
    "- **Maintain Entity Consistency**: When extracting entities, it's vital to "
    "ensure consistency.\n"
    'If an entity, such as "John Doe", is mentioned multiple times in the text '
    'but is referred to by different names or pronouns (e.g., "Joe", "he"),'
    "always use the most complete identifier for that entity throughout the "
    'knowledge graph. In this example, use "John Doe" as the entity ID.\n'
    "Remember, the knowledge graph should be coherent and easily understandable, "
    "so maintaining consistency in entity references is crucial.\n"
    "## 4. Strict Compliance\n"
    "Adhere to the rules strictly. Non-compliance will result in termination."
)
custom_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        (
            "human",
            (
                "Tip: Make sure to answer in the correct format and do "
                "not include any explanations. "
                "Make sure to **strictly** add name property for each node."
                "Remember that some information may be saved as properties of nodes and relationships, think it through add few items as properties as you see fit."
                "Use the given format to extract information from the "
                "following input: {input}"
            ),
        ),
    ]
)

#### Updated the graph creation prompt to include the following:
- Nodes in the graph include a **name** property.
- Name property is made to be added stricty to the nodes.
- Reminded to add properties to the nodes and relationships whenever necessary since properties where not emphasized by the model.

In [9]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [10]:
graph_maker = LLMGraphTransformer(llm=llm_anthropic, node_properties=True, relationship_properties=True, strict_mode=True, prompt=custom_prompt)

- Setting node_properties and relationship_properties to True to ensure that the model adds properties to the nodes and relationships whenever necessary.
- Providing properties to be extracted ensures consistency, however, since our usecase it open-ended, we will not be providing any properties to be extracted.

In [11]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../sample/mango.pdf")
docs = loader.load()

Read file into documents, and then break it down into chunks considering the context size of LLMs.

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=100)
text_chunks = text_splitter.split_documents(docs)

In [13]:
len(text_chunks)

7

In [18]:
graph_documents = graph_maker.convert_to_graph_documents(text_chunks)

The above step takes in our text chunks and returns a list GraphDocument object containing the nodes and relationships. Since graph_maker object was given custom prompt graph would be made as per our specification.

In [26]:
graph_documents[0].nodes
graph_documents[0].relationships

[Relationship(source=Node(id='Mango', type='Fruit'), target=Node(id='Mangifera Indica', type='Plant'), type='PRODUCED_BY'),
 Relationship(source=Node(id='Mangifera Indica', type='Plant'), target=Node(id='Mango_Fruit', type='Fruit'), type='PRODUCES'),
 Relationship(source=Node(id='Mangifera Indica', type='Plant'), target=Node(id='Mango_Leaf', type='Plant_part'), type='HAS_PART'),
 Relationship(source=Node(id='Mangifera Indica', type='Plant'), target=Node(id='Mango_Flower', type='Plant_part'), type='HAS_PART'),
 Relationship(source=Node(id='Bangladesh', type='Country'), target=Node(id='Mangifera Indica', type='Plant'), type='NATIONAL_TREE')]

In [14]:
from langchain_community.graphs import Neo4jGraph

In [15]:
graph = Neo4jGraph(driver_config={"max_connection_lifetime": 3600})

This creates a neo4j driver wrapper to interact with the neo4j database, provided by Langchain, we then use it write to our database.

In [19]:
graph.add_graph_documents(graph_documents)

Refreshing the graph schema would show us the relationships and nodes created. We will also be give to the LLMs to create cypher queries to extract information from the graph.

In [15]:
graph.refresh_schema()
graph.structured_schema

{'node_props': {'Fruit': [{'property': 'id', 'type': 'STRING'},
   {'property': 'name', 'type': 'STRING'},
   {'property': 'national_fruit_of', 'type': 'STRING'},
   {'property': 'scientific_name', 'type': 'STRING'},
   {'property': 'origin', 'type': 'STRING'},
   {'property': 'length_range', 'type': 'STRING'},
   {'property': 'weight_range', 'type': 'STRING'},
   {'property': 'ripening_time', 'type': 'STRING'}],
  'Plant': [{'property': 'id', 'type': 'STRING'},
   {'property': 'name', 'type': 'STRING'},
   {'property': 'crown_radius', 'type': 'STRING'},
   {'property': 'common_name', 'type': 'STRING'},
   {'property': 'lifespan', 'type': 'STRING'},
   {'property': 'height', 'type': 'STRING'},
   {'property': 'scientific_name', 'type': 'STRING'}],
  'Plant_part': [{'property': 'id', 'type': 'STRING'},
   {'property': 'name', 'type': 'STRING'},
   {'property': 'arrangement', 'type': 'STRING'},
   {'property': 'shape', 'type': 'STRING'},
   {'property': 'length', 'type': 'STRING'},
   {'

We now will use neo4jupyter package to visualize the graph. It is made on top of vis.js and is good for visualizing nep4j graphs in jupyter notebooks.

In [16]:
import neo4jupyter
neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

In [17]:
def show_graph():
    load_dotenv()
    NEO4J_URI = os.getenv("NEO4J_URI")
    NEW_NEO4J_URI = NEO4J_URI.replace("neo4j+s://", "bolt+s://")
    os.environ.pop('NEO4J_URI')

    from py2neo import Graph
    vis_graph = Graph(NEW_NEO4J_URI, auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD")))

    query = "MATCH (n) RETURN COUNT(n) AS total_nodes"
    result = vis_graph.run(query).data()

    total_nodes = result[0]['total_nodes']

    os.environ["NEO4J_URI"] = NEO4J_URI
    return neo4jupyter.draw(vis_graph,{}, limit=total_nodes)
    

This function will show us the graph, there is a workaround done in the function. 
It requires p2neo package to read from the neo4j database, not the Graph in py2neo reads neo4j credentials and uri directly from the environment if it exist and cannot be overridden. However, the issue is that it only support bolt protocol and the neo4j uri in environment is in neo4j+s protocol. So, we poped the uri from the environment and rewrote after visualization. 

In [18]:
show_graph()

After visualization, we see that there has been a lot of nodes that are similar with slight variations, this leads to disjointed nodes or nodes that cannot be reasoned by the LLMs. We will now merge the nodes that are similar to each other. We will do disambiguation with a custom prompt.

In [19]:

system_prompt = """
Act as a entity disambiugation tool and tell me which values reference the same entity. 
For example if I give you

Birds
Bird
Ant

You return to me

Birds, 1
Bird, 1
Ant, 2

As the Bird and Birds values have the same integer assigned to them, it means that they reference the same entity.

"""

disambiguate_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        (
            "human",
            (                
                "Perform disambiguation on the following values: \n{input}"
            ),
        ),
    ]
)

We now define pydantic classes to define the return types of the function calling feature in Claude/ OpenAI LLMs.

In [20]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class DisambiguatedNode(BaseModel):
    """Get the node name and id from the collection of nodes"""
    name: str = Field(..., title="Disambiquated name for the entity")
    id: int = Field(..., title="Disambiquated id for the entity")

class DisambiguatedNodeList(BaseModel):
    """Get the node name and id from the collection of nodes"""
    nodes: List[DisambiguatedNode] = Field(..., title="List of disambiguated nodes")

In [21]:
llm_openai = ChatOpenAI(model="gpt-4-turbo", temperature=0)

In [22]:
disambiguation_chain = disambiguate_prompt | llm_openai.with_structured_output(DisambiguatedNodeList)

We created a disambiguation chain that takes the prompt and runs it through LLM, and returns returns the disambiguated nodes after receiving the structured output as per out definition for function calling.

In [27]:
node_names = graph.query("MATCH (n) WHERE NOT n:Chunk RETURN n.name as name")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Chunk)} {position: line: 1, column: 23, offset: 22} for query: 'MATCH (n) WHERE NOT n:Chunk RETURN n.name as name'


Note: We ignore nodes with label Chunk, as those are used to store embedding chunks, we haven't used them in this case, but may exist in the db from a different operation.

In [35]:
node_names[:5]

[{'name': 'Mango'},
 {'name': 'Mangifera indica'},
 {'name': 'Mango fruit'},
 {'name': 'Mango leaf'},
 {'name': 'Mango flower'}]

In [40]:
node_names_list = [name["name"] for name in node_names]

In [41]:
node_names_list[:5]

['Mango', 'Mangifera indica', 'Mango fruit', 'Mango leaf', 'Mango flower']

In [42]:
response = disambiguation_chain.invoke({"input": "\n".join(node_names_list)})

We invoked the disambiguation chain on the node names and assigned same id to the nodes that are similar to each other.

In [45]:
response.dict()["nodes"][:10]

[{'name': 'Mango', 'id': 1},
 {'name': 'Mangifera indica', 'id': 1},
 {'name': 'Mango fruit', 'id': 1},
 {'name': 'Mango leaf', 'id': 1},
 {'name': 'Mango flower', 'id': 1},
 {'name': 'Bangladesh', 'id': 2},
 {'name': 'Carabao', 'id': 3},
 {'name': 'Langra', 'id': 1},
 {'name': 'Mango Pit', 'id': 1},
 {'name': 'Mango Seed', 'id': 1}]

In [46]:
node_clusters = [[node["name"], node["id"]] for node in response.dict()["nodes"]]

In [47]:
node_clusters[:10]

[['Mango', 1],
 ['Mangifera indica', 1],
 ['Mango fruit', 1],
 ['Mango leaf', 1],
 ['Mango flower', 1],
 ['Bangladesh', 2],
 ['Carabao', 3],
 ['Langra', 1],
 ['Mango Pit', 1],
 ['Mango Seed', 1]]

In [48]:
cluster_map = {}
for cluster_name, cluster_id in node_clusters:
    if cluster_id not in cluster_map:
        cluster_map[cluster_id] = cluster_name.capitalize()

In [50]:
mapped_nodes = []
for cluster_name, cluster_id in node_clusters:
    mapped_nodes.append([cluster_name,cluster_map[cluster_id]])

We created the name to which the nodes of the same clusters are to be renamed to.

In [51]:
mapped_nodes[:10]

[['Mango', 'Mango'],
 ['Mangifera indica', 'Mango'],
 ['Mango fruit', 'Mango'],
 ['Mango leaf', 'Mango'],
 ['Mango flower', 'Mango'],
 ['Bangladesh', 'Bangladesh'],
 ['Carabao', 'Carabao'],
 ['Langra', 'Mango'],
 ['Mango Pit', 'Mango'],
 ['Mango Seed', 'Mango']]

In [54]:
node_rename_cypher = """
UNWIND $mapping as pair
MATCH (n {name: pair[0]})
SET n.name = pair[1]
"""


graph.query(node_rename_cypher,{
    "mapping": mapped_nodes
})

[]

We renamed the nodes in the db

In [58]:
node_merge_cypher = """
MATCH (n)
WHERE NOT 'Chunk' IN labels(n)
WITH n.name as nodeId, collect(n) as nodes
CALL apoc.refactor.mergeNodes(nodes, {properties: "combine", mergeRels: true})
YIELD node
RETURN node;
"""

graph.query(node_merge_cypher)

[{'node': {'length_range': '5-25 centimetres',
   'embryony': 'monoembryonic',
   'thickness': '1–2 mm',
   'lifespan': 'Over 300 years',
   'taste': 'Sour',
   'origin': ['Region between northwestern Myanmar, Bangladesh, and northeastern India',
    "seedling of 'Haden'",
    'India'],
   'description': ["important export product, considered 'the king of mangoes'",
    'prolific cultivar in Jamaica',
    'Thick juice made of mangoes with sugar or milk',
    'Fruit preserves',
    'Sweet, grated mango delicacy',
    'Dried and powdered unripe mango'],
   'type': ['Evergreen', 'Indian type', 'Summer drink', 'Jam', 'Pickle'],
   'arrangement': ['Alternate', 'Terminal panicles'],
   'petal_count': 'Five',
   'id': ['Mango',
    'Mangifera Indica',
    'Mango_Fruit',
    'Mango_Leaf',
    'Mango_Flower',
    'Langra',
    'Mango_Pit',
    'Mango_Seed',
    'Mango_Tree',
    'Alphonso',
    'Julie',
    'Tommy Atkins',
    'Alphonso Mango',
    'Ataulfo Mango',
    'Mango Juice',
    'Mango

We now merge the nodes with same name property, this is why we forced the LLM earlier to add name property, any arbitrary property name could be chose, name happened to be easier for the LLM to extract.

In [59]:
graph.refresh_schema()
graph.structured_schema

{'node_props': {'Fruit': [{'property': 'id', 'type': 'LIST'},
   {'property': 'name', 'type': 'STRING'},
   {'property': 'national_fruit_of', 'type': 'STRING'},
   {'property': 'scientific_name', 'type': 'STRING'},
   {'property': 'origin', 'type': 'LIST'},
   {'property': 'crown_radius', 'type': 'STRING'},
   {'property': 'common_name', 'type': 'STRING'},
   {'property': 'lifespan', 'type': 'STRING'},
   {'property': 'height', 'type': 'STRING'},
   {'property': 'length_range', 'type': 'STRING'},
   {'property': 'weight_range', 'type': 'STRING'},
   {'property': 'ripening_time', 'type': 'STRING'},
   {'property': 'arrangement', 'type': 'LIST'},
   {'property': 'shape', 'type': 'STRING'},
   {'property': 'length', 'type': 'LIST'},
   {'property': 'width', 'type': 'STRING'},
   {'property': 'type', 'type': 'LIST'},
   {'property': 'fragrance', 'type': 'STRING'},
   {'property': 'petal_count', 'type': 'STRING'},
   {'property': 'petal_length', 'type': 'STRING'},
   {'property': 'panicle_l

In [60]:
show_graph()

We note provide a custom prompt to the LLM to create a cypher query to extract information from the graph, we provide key insights to the LLM here:

- When the node names where rewritter we used capitalization to make the nodes more readable and to make consistent query on the graph.
- Remind the LLm to use properties to extract information from the graph.
- We ask it use synonyms to extract information from the graph.

In [62]:
CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.
Instructions:
- Use only the provided relationship types and properties in the schema.
- Do not use any other relationship types or properties that are not listed.
- Examine the properties of nodes and relationships closely, as the answer might be found there.
- STRICTLY use **capitalize** when using **name** property of nodes, for example instead of "mango" use "Mango" or "Mango Tree" use "Mango tree".
- When fetching from properties use synonyms of values to match, for example instead of just Leaf do an or on Mango Leaf".

Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [63]:
from langchain.chains import GraphCypherQAChain

We create a graph query chain that interprets the user queries, create cypher queries and runs them on the graph, and uses LLM to creat human readable responses.

In [66]:
graph_qa_chain = GraphCypherQAChain.from_llm(
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    validate=True,    
)

In [67]:
graph_qa_chain.invoke(
    "what is the scientific name of mango?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Mango {name: "Mango"})
RETURN m.scientific_name AS ScientificName

Full Context:
[{'ScientificName': 'Mangifera indica'}]

> Finished chain.


{'query': 'what is the scientific name of mango?',
 'result': 'The scientific name of mango is Mangifera indica.'}

In [68]:
graph_qa_chain.invoke(
    "what is the length of mango?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Mango)
WHERE m.name = "Mango"
RETURN m.length_range AS LengthOfMango

Full Context:
[{'LengthOfMango': '5-25 centimetres'}]

> Finished chain.


{'query': 'what is the length of mango?',
 'result': 'The length of a mango can range from 5 to 25 centimetres.'}

In [73]:
graph_qa_chain.invoke(
    "what is the height of mango?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Mango {name: "Mango"})
RETURN m.height AS Height

Full Context:
[{'Height': '30-40 metres'}]

> Finished chain.


{'query': 'what is the height of mango?',
 'result': 'The height of a mango tree typically ranges from 30 to 40 metres.'}

In [74]:
graph_qa_chain.invoke(
    "where are mangoes consumed?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Mango)-[:CONSUMED_IN]->(l:Location)
RETURN m.name AS Mango, l.name AS Location

Full Context:
[{'Mango': 'Mango', 'Location': 'Southeast asia'}, {'Mango': 'Mango', 'Location': 'South asia'}, {'Mango': 'Mango', 'Location': 'Philippines'}, {'Mango': 'Mango', 'Location': 'Central america'}]

> Finished chain.


{'query': 'where are mangoes consumed?',
 'result': 'Mangoes are consumed in Southeast Asia, South Asia, the Philippines, and Central America.'}

In [75]:
graph_qa_chain.invoke(
    "mango is the national fruit of which all countries?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Mango {name: "Mango"})-[:NATIONAL_FRUIT_OF]->(c:Country)
RETURN c.name AS CountryName

Full Context:
[{'CountryName': 'India'}]

> Finished chain.


{'query': 'mango is the national fruit of which all countries?',
 'result': 'Mango is the national fruit of India.'}

In [76]:
graph_qa_chain.invoke(
    "mango is the national fruit/tree of which all countries?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Mango {name: "Mango"})-[:NATIONAL_FRUIT_OF|:NATIONAL_TREE_OF]->(c:Country)
RETURN c.name AS Country



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':NATIONAL_FRUIT_OF|NATIONAL_TREE_OF' instead)} {position: line: 2, column: 52, offset: 58} for query: 'cypher\nMATCH (m:Mango {name: "Mango"})-[:NATIONAL_FRUIT_OF|:NATIONAL_TREE_OF]->(c:Country)\nRETURN c.name AS Country\n'


Full Context:
[{'Country': 'India'}, {'Country': 'Bangladesh'}]

> Finished chain.


{'query': 'mango is the national fruit/tree of which all countries?',
 'result': 'Mango is the national fruit of India and Bangladesh.'}

In [77]:
graph_qa_chain.invoke(
    "What all does mango contain?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Mango {name: "Mango"})-[:CONTAINS]->(c)
RETURN c.name AS Contains

Full Context:
[{'Contains': 'Vitamin c'}, {'Contains': 'Folate'}, {'Contains': 'Mango'}]

> Finished chain.


{'query': 'What all does mango contain?',
 'result': 'Mango contains Vitamin C and Folate.'}

In [78]:
graph_qa_chain.invoke(
    "where all are mangoes produced?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Mango)-[:PRODUCED_BY]->(p)
RETURN p.name AS Producer, p.location AS Location

Full Context:
[{'Producer': 'Mango', 'Location': 'southern Florida'}]

> Finished chain.


{'query': 'where all are mangoes produced?',
 'result': 'Mangoes are produced in southern Florida.'}

In [82]:
graph_qa_chain.invoke(
    "where all countries produces mangoes?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Country)-[:PRODUCES]->(m:Mango)
RETURN c.name AS Country, collect(m.name) AS Mangoes

Full Context:
[{'Country': 'Brazil', 'Mangoes': ['Mango']}, {'Country': 'Mexico', 'Mangoes': ['Mango']}, {'Country': 'Spain', 'Mangoes': ['Mango']}, {'Country': 'India', 'Mangoes': ['Mango']}, {'Country': 'Indonesia', 'Mangoes': ['Mango']}, {'Country': 'China', 'Mangoes': ['Mango']}, {'Country': 'Pakistan', 'Mangoes': ['Mango']}, {'Country': 'Australia', 'Mangoes': ['Mango']}]

> Finished chain.


{'query': 'where all countries produces mangoes?',
 'result': 'Brazil, Mexico, Spain, India, Indonesia, China, Pakistan, and Australia all produce mangoes.'}